In [1]:
import numpy as np
import csv
import torch

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('/data/rgur/courses/cs_7643_deep_learning/hw4/')

# Just run this block. Please do not modify the following code.
import math
import time

# Pytorch package
import torch
import torch.nn as nn
import torch.optim as optim

# Torchtest package
from torchtext.datasets import Multi30k
from torchtext.data import Field, BucketIterator, Example, TabularDataset, interleave_keys, LabelField

# Tqdm progress bar
from tqdm import tqdm_notebook, tqdm

# Code provide to you for training and evaluation
#from hw4_code.utils import train, evaluate, set_seed_nb, unit_test_values
from utils import qe_train, qe_evaluate

import importlib

# Check device availability
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("You are using device: %s" % device)

You are using device: cuda


In [2]:
# You don't need to modify any code in this block

# Define the maximum length of the sentence. Shorter sentences will be padded to that length and longer sentences will be croped. Given that the average length of the sentence in the corpus is around 13, we can set it to 20
MAX_LEN = 20

# Define the source and target language
SRC = Field(tokenize = "spacy",
            tokenizer_language="ro_core_news_sm",
            init_token = '<sos>',
            eos_token = '<eos>',
            fix_length = MAX_LEN,
            lower = True)

TRG = Field(tokenize = "spacy",
            tokenizer_language="en",
            init_token = '<sos>',
            eos_token = '<eos>',
            fix_length = MAX_LEN,
            lower = True)

Z = LabelField(dtype=torch.float, batch_first=True, use_vocab=False)

train_data = TabularDataset(path='/data/rgur/courses/cs_7643_deep_learning/project/mlqe/data/ro-en/train.roen.df.short.tsv',format='TSV',fields={'original':('src',SRC),
'translation':('trg',TRG),
'z_mean':('z',Z)})

val_data = TabularDataset(path='/data/rgur/courses/cs_7643_deep_learning/project/mlqe/data/ro-en/dev.roen.df.short.tsv',format='TSV',fields={'original':('src',SRC),
'translation':('trg',TRG),
'z_mean':('z',Z)})

# Define Batchsize
BATCH_SIZE = 128

# Build the vocabulary associated with each language
SRC.build_vocab(train_data, min_freq = 2)
TRG.build_vocab(train_data, min_freq = 2)

# Get the padding index to be ignored later in loss calculation
PAD_IDX = TRG.vocab.stoi['<pad>']

# Get data-loaders using BucketIterator
train_loader = BucketIterator( #Defines an iterator that batches examples of similar lengths together.
    train_data,
    batch_size = BATCH_SIZE, device = device, sort_key=lambda x: interleave_keys(len(x.src), len(x.trg)))

val_loader = BucketIterator( #Defines an iterator that batches examples of similar lengths together.
    val_data,
    batch_size = BATCH_SIZE, device = device, sort_key=lambda x: interleave_keys(len(x.src), len(x.trg)))

# Get the input and the output sizes for model
input_size = len(SRC.vocab)
output_size = len(TRG.vocab)

/home/rgur/.conda/envs/cs7643-a2/lib/python3.8/site-packages/torchtext/data/field.py:150: UserWarning: Field class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)
/home/rgur/.conda/envs/cs7643-a2/lib/python3.8/site-packages/torchtext/data/field.py:150: UserWarning: Field class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)
/home/rgur/.conda/envs/cs7643-a2/lib/python3.8/site-packages/torchtext/data/field.py:150: UserWarning: LabelField class will be retired soon and moved t

In [3]:
class QEModel(nn.Module):
    """
    A single-layer Transformer which encodes a sequence of text and 
    performs binary classification.

    The model has a vocab size of V, works on
    sequences of length T, has an hidden dimension of H, uses word vectors
    also of dimension H, and operates on minibatches of size N.
    """
    def __init__(self, input_size=256*20):
        super(QEModel, self).__init__()
        self.lin1 = nn.Linear(256*20,1028)
        self.lin2 = nn.Linear(1028,1)
    
    def forward(self, x):
        x = nn.functional.relu( self.lin1(x) )
        x = nn.functional.dropout(x,p=.2)
        return self.lin2(x)

In [4]:
qe_model = QEModel().to(device)

In [5]:
#importlib.reload(hw4_code.models.Transformer)
from hw4_code.models.Transformer import TransformerTranslator
source_model = TransformerTranslator(output_size, input_size, device, max_length = MAX_LEN).to(device)
target_model = TransformerTranslator(input_size, output_size, device, max_length = MAX_LEN).to(device)

In [6]:
source_model.load_state_dict(torch.load('/data/rgur/courses/cs_7643_deep_learning/project/en_ro.pt'))
target_model.load_state_dict(torch.load('/data/rgur/courses/cs_7643_deep_learning/project/ro_en.pt'))

<All keys matched successfully>

In [7]:
# Hyperparameters
learning_rate = .001
EPOCHS = 25

# Model


# optimizer = optim.Adam(model.parameters(), lr = learning_rate)
optimizer = torch.optim.Adam(qe_model.parameters(), lr=learning_rate)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer)
criterion = nn.MSELoss()

In [13]:
for epoch_idx in range(EPOCHS):
    print("-----------------------------------")
    print("Epoch %d" % (epoch_idx+1))
    print("-----------------------------------")
    
    train_loss, avg_train_loss = qe_train(qe_model, source_model, target_model, train_loader, optimizer, criterion, scheduler = None)
    train_loss = train_loss.item()
    scheduler.step(train_loss)

    #val_loss, avg_val_loss = evaluate(trans_model, val_loader, criterion)

    avg_train_loss = avg_train_loss.item()
    val_loss, avg_val_loss, r2, mae = qe_evaluate(qe_model, source_model, target_model, val_loader, criterion)
    #avg_val_loss = 0
    avg_val_loss = avg_val_loss.item()

    print("Training Loss: %.4f. Validation RMSE: %.4f. Validation R2: %.4f. Validation MAE %.4f" % (np.sqrt(avg_train_loss), np.sqrt(avg_val_loss), r2, mae))

-----------------------------------
Epoch 1
-----------------------------------


  0%|          | 0/53 [00:00<?, ?it/s]

  0%|          | 0/8 [00:00<?, ?it/s]

preds: tensor([ 0.7939, -0.8459,  0.0559, -0.0794,  0.8472], device='cuda:0',
       grad_fn=<SliceBackward>)
label: tensor([0.8464, 0.6517, 0.8446, 0.8611, 0.5942], device='cuda:0')
Training Loss: 0.5842. Validation Loss: 0.9382. 
-----------------------------------
Epoch 2
-----------------------------------


  0%|          | 0/53 [00:00<?, ?it/s]

...

  0%|          | 0/8 [00:00<?, ?it/s]

preds: tensor([-0.9348,  1.0479, -0.0822, -0.5423,  1.3441], device='cuda:0',
       grad_fn=<SliceBackward>)
label: tensor([-1.8307,  1.0561, -1.0570, -1.6819,  1.0055], device='cuda:0')
Training Loss: 0.5254. Validation Loss: 1.0184. 
-----------------------------------
Epoch 25
-----------------------------------


  0%|          | 0/53 [00:00<?, ?it/s]

  0%|          | 0/8 [00:00<?, ?it/s]

preds: tensor([-0.6575,  0.1499,  0.1089,  0.3750,  0.3917], device='cuda:0',
       grad_fn=<SliceBackward>)
label: tensor([-0.2678, -0.3062, -0.8595,  0.1183, -0.3075], device='cuda:0')
Training Loss: 0.5189. Validation Loss: 1.0658. 
